# Step 1: Exchange pairs

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution
import time

In [124]:
df_matched = pd.read_stata('sample_matched_NY.dta')
df_unmatched = pd.read_stata('sample_unmatched_NY.dta')

'''
df_matched = pd.read_stata('test_sample_matched.dta')
df_unmatched = pd.read_stata('sample_unmatched.dta')
'''
def Exchange_pairs(df_matched, df_unmatched):
    # t1 = time.time()
    
    df_matched_column = df_matched.columns
    df_matched.columns = df_matched_column + '1m'
    df_unmatched.columns = df_unmatched.columns.str.replace('lender_id', 'lender_id1m')
    df1 = pd.merge(df_matched, df_unmatched, on = 'lender_id1m', how = 'inner')
    
    l = df1.columns[:-6].append([df1.columns[-6:] + '1um'])
    df1.columns = l
    df_matched.columns = df_matched_column
    df2 = pd.merge(df_matched, df1, left_on = 'loan_id', right_on = 'loan_id1um', how = 'inner') 
    
    ll = (df2.columns[:7]+'2m').append(df2.columns[7:])
    df2.columns = ll
    df_unmatched.columns = df_unmatched.columns.str.replace('lender_id1m', 'lender_id')
    df3 = pd.merge(df_unmatched, df2, left_on = ['lender_id','loan_id'], right_on = ['lender_id2m','loan_id1m'], how = 'inner')
    lll = (df3.columns[:7]+'2um').append(df3.columns[7:])
    df3.columns = lll
    
    df_keep = pd.DataFrame()
    for i in range(1, 6):
        name = "value" + str(i)
        df_keep[name] = df3["var"+str(i)+"1m"] + df3["var"+str(i)+"2m"] - df3["var"+str(i)+"1um"] - df3["var"+str(i)+"2um"]
    # t2 = time.time()
    # print("Running time: ", t2-t1)
    return df_keep
df_keep = Exchange_pairs(df_matched, df_unmatched)
df_keep

,value1,value2,value3,value4,value5
0,1.0,-128.862122,95195504.0,0.000000,0.0
1,1.0,-123.937958,96773328.0,0.000000,0.0
2,0.0,94.940582,97036304.0,0.000000,0.0
3,0.0,-97.312332,23914.0,3.413793,0.0
4,0.0,-97.312332,0.0,0.413793,1.0
...,...,...,...,...,...
11222911,0.0,-163.896729,-27316.0,2.000000,-1.0
11222912,0.0,-163.896729,109264.0,2.600000,-1.0
11222913,0.0,-152.651489,-54632.0,3.000000,0.0
11222914,0.0,-31.092916,-109264.0,4.000000,-1.0


# Step 2: Define objective function 

Step 2: Define objective function -- (P15 equation 7 in Fox (2018) or P818 equation 4 in Schwert (2018) )
define a function that calculates the score of the matching model for given parameters
i.e. count how many inequalities are satisfied under given parameters 
objective function (score) =sum of 1(inequalities satisfied), where
$$Inequalities = beta \times value \geqslant 0$$
$\beta$ is the parameter vector
Value vector for a given data point (i.e. value 1-6 for a given data point)
Hint: apply (multiply) the parameter vector to the data matrix (transposed) and then count the non-negative values

The objective function $$Q(\beta) = \Sigma \mathbb{1}[X_{N\times 6}\beta_{6\times 1}]$$

# Step 3: Differential Evolution

In [64]:
# use differential_evolution in scipy.optimize to find the parameters than maximize the score calculated in step 2
# Reference
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html
# https://pablormier.github.io/2017/09/05/a-tutorial-on-differential-evolution-with-python/

In [18]:
df_test = df_keep.iloc[: 5, :]
df_test

,value1,value2,value3,value4,value5
0,1.0,-128.862122,95195504.0,0.000000,0.0
1,1.0,-123.937958,96773328.0,0.000000,0.0
2,0.0,94.940582,97036304.0,0.000000,0.0
3,0.0,-97.312332,23914.0,3.413793,0.0
4,0.0,-97.312332,0.0,0.413793,1.0


In [161]:
# 1. When input, make sure df contains all the values you need
# 2. specify df = X in def objectfunc(beta, df = df_test) before you run differential_evolution
def objectfunc(beta, df = df_keep):
    '''
    num = df.shape[1] # This is the num of Parameters( dimension of beta)
    obj = 0
    for i in range(0, num):
        obj = obj + np.array(df.iloc[:, i]) * beta[i]
        # print(obj)
    return -sum(obj >= 0)
    '''
    return -sum(df.dot(beta) >=0 )

In [162]:
# This is a test for debugging
beta = np.array([1, 2, 3, 4, 5])
obj = objectfunc(beta, df_keep)
obj

-5908089

In [163]:
# This is a test for debugging
beta = np.array([1, 2, 3, 4, 5])
obj = objectfunc(beta, df_test)
obj

-4

In [115]:
# specify bounds of every parameters
'''
num = df_test.shape[0]
num = df_test.shape[0]
bounds = [(-100, 100)] * num
'''
t1 = time.time()
bounds = [(1, 1.000000001), (-100, 100), (-100, 100), (-100, 100), (-100, 100)]
result = differential_evolution(objectfunc, bounds)
print(result)
t2 = time.time()
print("Running time: ", t2-t1)

     fun: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 606
     nit: 7
 success: True
       x: array([  1.        ,  52.72167492, -88.99536062,  15.67398543,
        27.12052265])
Running time:  0.19236969947814941


# Step 4: Use subsampling to construct confidence intervals

The idea is to select some random subsamples (100-200) that mimics the original sample and re-do the step1-3 and get another estimated parameter. Then calculate the 5% and 95% percentile of the 100 (200) parameters.
To mimic the original sample, we choose the subsample for fixed share for each state.

In [122]:
# The random sampling ratio = 10% here
def Subsample(df, ratio):
    # t1 = time.time()
    df_sample = df.loc[np.random.choice(df.index, round(len(df) * ratio), replace = False)]
    # t2 = time.time()
    # print("Running time: ", t2-t1)
    return df_sample
df_matched_sample = Subsample(df_matched, 0.1)
df_unmatched_sample = Subsample(df_unmatched, 0.1)

In [148]:
# num is the # of random samples(or the # of simulations)
# ratio is the random sampling ratio, which is 10% here
def Simulation(num, ratio, df_matched, df_unmatched):
    df_result = pd.DataFrame()
    for i in range(0, num):
        df_matched_sample = Subsample(df_matched, ratio)
        df_unmatched_sample = Subsample(df_unmatched, ratio)
        df_keep = Exchange_pairs(df_matched_sample, df_unmatched_sample)
        bounds = [(1, 1.000000001), (-100, 100), (-100, 100), (-100, 100), (-100, 100)] # fix beta_1 = 1
        result = differential_evolution(objectfunc, bounds)
        df_result = df_result.append(pd.Series(result.x), ignore_index = True)
        
    
    return df_result


t1 = time.time()
df_result = Simulation(200, 0.1, df_matched, df_unmatched)
t2 = time.time()
print("Simulation time: ", t2-t1)
df_result

Simulation time:  54.844584465026855


,0,1,2,3,4
0,1.0,97.282217,-50.369656,47.810394,-22.267439
1,1.0,8.647756,-10.767369,4.020486,33.549854
2,1.0,46.918643,-5.801916,-21.076612,42.511858
3,1.0,77.949363,-26.799431,-19.884129,64.419495
4,1.0,69.911933,-73.676332,22.729545,-16.535253
...,...,...,...,...,...
195,1.0,52.989474,-52.396316,6.188839,97.536375
196,1.0,42.337933,-45.487276,13.210631,-12.301303
197,1.0,85.400034,-22.133411,-65.623209,92.908057
198,1.0,51.272742,-45.118289,34.067582,-96.971592


In [160]:
print("The 5% quantile of parameters are")
print(df_result.quantile(0.05))
print("The 95% quantile of parameters are")
print(df_result.quantile(0.95))

The 5% quantile of parameters are
0     1.000000
1     6.783120
2   -91.603813
3   -74.832412
4   -88.183530
Name: 0.05, dtype: float64
The 95% quantile of parameters are
0     1.000000
1    96.682176
2    -6.704712
3    87.780066
4    84.220047
Name: 0.95, dtype: float64


# Step 5: Model Fitness

1) Number of inequalities satisfied: count how many inequalities are satisfied under the optimized parameter choice.

2) In sample prediction: assign firms to lenders according to the estimated value function and count how many of the observed (P820, footnote 51 – Schwert (2018))
Target Output – Table IX X in Schwert (2018)